In [1]:
import tensorflow as tf

In [2]:
EPOCHS = 30
BATCH_SIZE = 32
LR = 1e-5
CLASS_NUMBER =3
IMAGE_SIZE = 224
MODEL_NAME = ''
dataset_folder = 'dataset/train'

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
)

train_generator = datagen.flow_from_directory(
    dataset_folder,
    target_size= (IMAGE_SIZE,IMAGE_SIZE),
    batch_size= BATCH_SIZE,
    subset= 'training'
)

val_generator = datagen.flow_from_directory(
    dataset_folder,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

print(train_generator.class_indices)

In [4]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))

In [5]:
IMAGE_SHAPE = (IMAGE_SIZE,IMAGE_SIZE,3)

In [6]:
base_model = tf.keras.applications.ResNet50(
    input_shape=IMAGE_SHAPE,
    include_top=False,
    weights='imagenet' 
)
base_model.trainable = False

In [7]:
model = tf.keras.Sequential([ 
  base_model,
  tf.keras.layers.Conv2D(32,3, activation = 'relu'), 
  tf.keras.layers.Dropout(0.2), 
  tf.keras.layers.GlobalAveragePooling2D(), 
  tf.keras.layers.Dense(CLASS_NUMBER,activation='softmax')
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=LR),
    loss= 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss' ,patience=2, mode='min',verbose=1),
    tf.keras.callbacks.ModelCheckpoint(filepath='models/2haziran__model.{epoch:02d}-{val_loss:.5f}.h5',monitor='val_loss',mode='min',save_best_only=True),
]

history = model.fit(
    train_generator, 
    epochs = EPOCHS, 
    callbacks=[my_callbacks],
    validation_data=val_generator,
)

model.summary()

In [ ]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir) 

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) 
tflite_model = converter.convert() 

In [10]:
with open(MODEL_NAME + '.tflite', 'wb') as f:
    f.write(tflite_model)

In [11]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), history.history["val_accuracy"], label="val_acc")
plt.title("LİVENESS DETECTİON DATASET")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(MODEL_NAME)